In [13]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
from networks import Encoder, Decoder, MetaModel, TrainingCallback
import numpy as np
import ast
import matplotlib.pyplot as plt
from sindy import SINDy


x, x_dot = np.load("datalorenz/paper2048.npy").astype(np.float32)
x = tf.reshape(x,[-1,128])
x_dot = tf.reshape(x,[-1,128])

gpus_google=tf.config.list_physical_devices("GPU")
gpus_pic=tf.config.list_physical_devices("XLA_GPU")
gpus = gpus_pic
for k in gpus_google:
    gpus.append(k)
    
    
encoder = Encoder()
decoder = Decoder()
sindy = SINDy()
models = [encoder, decoder, sindy]
    
if len(gpus)>0:
    metamodel = MetaModel(models, total_epochs=int(1e4+1e3), when_zero_lambda3=1000)
    with tf.device("GPU"):
        history = metamodel.fit(x=x, y=x_dot, epochs=metamodel.total_epochs, batch_size=8000,
                        callbacks=[TrainingCallback(), tf.keras.callbacks.TensorBoard("logs/prueba")])
else:
    metamodel = MetaModel(models, total_epochs=2750, when_zero_lambda3=400)
    history = metamodel.fit(x=x, y=x_dot, epochs=int(metamodel.total_epochs), batch_size=2000,
                callbacks=[TrainingCallback(), tf.keras.callbacks.TensorBoard("logs/chiquitito")])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
